# Identify a CPU bottleneck caused by a callback process with Amazon SageMaker Debugger 

***Note: 본 노트북 코드는 [영문 노트북](https://github.com/aws/amazon-sagemaker-examples/blob/master/sagemaker-debugger/tensorflow_profiling/callback_bottleneck.ipynb)을 한국어화(중간중간 역주 추가와 documentation 보완)하면서 보완하면서, Debugger 예제 코드를 추가하였습니다.***

이 노트북에서는 TensorFlow Keras 콜백으로 인해 발생하는 훈련 병목 현상을 식별하는 방법을 보여줍니다. 이러한 유형의 병목 현상을 시뮬레이션하기 위해 Amazon SageMaker Debugger의 텐서 모니터링 기능과 관련된 콜백을 호출하여 많은 수의 텐서를 높은 빈도로 수집합니다.

### Install SageMaker and smdebug
2020년 12월에 출시된 새로운 디버거 프로파일링 기능을 사용하려면, 최신 버전의 SageMaker 및 SMDebug SDK가 설치되어 있는지 확인하세요. 다음 코드 셀을 사용하여 라이브러리를 업데이트하고 Jupyter 커널을 다시 시작하여 업데이트를 적용합니다.

In [ ]:
import sys
import IPython
import boto3    
import sagemaker    
install_needed = False  # should only be True once
if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U sagemaker smdebug
    IPython.Application.instance().kernel.do_shutdown(True)

bucket = sagemaker.Session().default_bucket()    

In [ ]:
# !pip install -qU horovod

<br>

## 1. Prepare training dataset
---

### Tensorflow Datasets package

우선 노트북 커널을 Tensorflow 2.x로 설정합니다.

이 실험에는 CIFAR-10 데이터셋을 사용합니다. CIFAR-10 데이터셋을 다운로드하고 TFRecord 형식으로 변환하려면 `demo/generate_cifar10_tfrecords`를 실행하고 tfrecord 파일을 S3 버킷에 업로드하세요.

In [ ]:
!python demo/generate_cifar10_tfrecords.py --data-dir=./data

In [ ]:
import sagemaker

s3_bucket = sagemaker.Session().default_bucket()

dataset_prefix = "data/cifar10-tfrecords"
desired_s3_uri = f"s3://{s3_bucket}/{dataset_prefix}"

dataset_location = sagemaker.s3.S3Uploader.upload(local_path="data", desired_s3_uri=desired_s3_uri)
print(f"Dataset uploaded to {dataset_location}")

<br>

## 2. Create a Training Job with Profiling Enabled<a class="anchor" id="option-1"></a>
---

표준 [SageMaker Estimator API for Tensorflow](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/sagemaker.tensorflow.html#tensorflow-estimator)를 사용하여 훈련 작업을 생성합니다. 프로파일링을 사용하려면 `ProfilerConfig` 객체를 만들고 `TensorFlow` estimator `profiler_config` 파라메터에 전달합니다. 본 예제는 프로파일링 간격을 500밀리초(0.5초)로 설정했습니다.

### Set a profiler configuration

In [ ]:
from sagemaker.debugger import ProfilerConfig, FrameworkProfile

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500,
    framework_profile_params=FrameworkProfile(
        local_path="/opt/ml/output/profiler/", start_step=5, num_steps=2
    ),
)

### Configure Debugger hook

50 step마다 텐서를 수집하도록 Debugger hook를 구성합니다.

In [ ]:
import os
from sagemaker.debugger import DebuggerHookConfig, CollectionConfig

debugger_hook_config = DebuggerHookConfig(
    hook_parameters={"save_interval": "50"},
    collection_configs=[
        CollectionConfig(name="outputs"),
        CollectionConfig(name="gradients"),
        CollectionConfig(name="weights"),
        CollectionConfig(name="layers"),
    ],
)

### Define hyperparameters

훈련 스크립트 [train_tf_bottleneck.py](./demo/train_tf_bottleneck) 여러 파라메터들을 허용합니다. Epoch 수 및 배치 크기와 같은 하이퍼파라메터를 정의합니다.

In [ ]:
hyperparameters = {"epoch": 2, "batch_size": 128}

### Get the image URI

이 노트북을 실행하는 리전에 따라 도커 이미지가 달라집니다.

In [ ]:
import boto3

session = boto3.session.Session()
region = session.region_name

image_uri = f"763104351884.dkr.ecr.{region}.amazonaws.com/tensorflow-training:2.3.1-gpu-py37-cu110-ubuntu18.04"

### Define SageMaker Tensorflow Estimator

프로파일링을 활성화하려면 디버거 프로파일링 구성 (`profiler_config`), 디버거 규칙 목록 (`rules`) 및 이미지 URI (`image_uri)`를 estimator에 전달해야 합니다. 디버거는 SageMaker estimator가 훈련 작업을 요청하는 동안 모니터링 및 프로파일링을 활성화합니다.

In [ ]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

job_name = "network-bottleneck"
instance_count = 1
instance_type = "ml.p2.xlarge"
entry_script = "train_tf_bottleneck.py"

estimator = TensorFlow(
    role=sagemaker.get_execution_role(),
    image_uri=image_uri,
    base_job_name=job_name,
    instance_type=instance_type,
    instance_count=instance_count,
    entry_point=entry_script,
    source_dir="demo",
    profiler_config=profiler_config,
    debugger_hook_config=debugger_hook_config,
    script_mode=True,
    hyperparameters=hyperparameters,
    input_mode="Pipe",
)

### Start training job

`wait=False` argument를 포함한 `estimator.fit()`은 백그라운드에서 훈련 작업을 시작합니다. 대시보드 또는 분석 노트북 실행을 계속할 수 있습니다.

In [ ]:
remote_inputs = {"train": dataset_location + "/train"}

estimator.fit(remote_inputs, wait=True)

In [ ]:
training_job_name = estimator.latest_training_job.name
print("Training Job Name:  {}".format(training_job_name))

AWS 콘솔 화면에서 `Training jobs`를 확인해 보세요. 아래 코드 셀에서 자동으로 생성되는 링크를 클릭하셔도 됩니다.

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job</a> After About 5 Minutes</b>'.format(
            region, training_job_name
        )
    )
)

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, training_job_name
        )
    )
)

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Training Job Has Completed</b>'.format(
            bucket, training_job_name, region
        )
    )
)

<br>

## 3. [SageMaker Studio Only] Monitor the system resource utilization using SageMaker Studio
---

SageMaker Studio는 시스템 및 프레임워크 성능 메트릭의 분석 리포트와 plot을 찾을 수 있는 Sagemaker Debugger용 시각화 도구를 제공합니다.

SageMaker Studio에서 이 정보에 액세스하려면, 왼쪽의 마지막 아이콘을 클릭하여 `SageMaker Components and registries` 를 열고 `Experiments and trials` 을 선택합니다. 훈련 작업 목록이 표시됩니다. 해당 job을 마우스 오른쪽 버튼으로 클릭하면 팝업 메뉴가 표시되고, `Open Debugger for insights` 를 클릭하면 아래와 같이 SageMaker 디버거에 대한 새 탭이 열립니다.

`Overview`와 `Nodes`의 두 가지 탭이 있습니다. `Overview`는 빠른 검토를 위한 프로파일링 요약을 제공하고, `Nodes`는 모든 노드에 대한 자세한 utilization 정보를 제공합니다.

<br>

## 4. SageMaker Debugger profiling analysis utilities
---

프로파일링 분석 유틸리티를 사용하여 문제의 원인에 대한 더 깊은 통찰력(insight)을 얻을 수 있습니다. 본 실습에서는 bokeh 및 smdebug 패키지를 사용합니다.

In [ ]:
! pip install bokeh==2.1.1
! pip install smdebug

smdebug를 사용하여 GPU 및 프레임워크 지표(metric)를 추출합니다.

In [ ]:
import boto3
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob
from smdebug.profiler.analysis.utils.profiler_data_to_pandas import PandasFrame


training_job_name = estimator.latest_training_job.name
region = boto3.Session().region_name

tj = TrainingJob(training_job_name, region)

pf = PandasFrame(tj.profiler_s3_output_path)

# extract gpu metrics
system_metrics_df = pf.get_all_system_metrics()
gpus = system_metrics_df[system_metrics_df["dimension"] == "GPUUtilization"]
timestamps = gpus["timestamp_us"].to_numpy()
values = gpus["value"].to_numpy()

# exctract framework metrics
framework_metrics_df = pf.get_all_framework_metrics(
    selected_framework_metrics=["Step:ModeKeys.TRAIN", "Step:ModeKeys.GLOBAL"]
)
train_steps = framework_metrics_df[
    framework_metrics_df["framework_metric"].isin(["Step:ModeKeys.TRAIN", "Step:ModeKeys.GLOBAL"])
]
start_step = train_steps["start_time_us"].to_numpy()
end_step = train_steps["end_time_us"].to_numpy()
step_num = train_steps["step"].to_numpy()

bokeh를 사용하여 GPU 지표와 훈련 진행 상황을 동일한 그래프에 표시합니다. 이를 통해 둘 사이의 상관 관계를 파악할 수 있습니다. GPU 사용률 감소가 노란색으로 표시된 50번째 step마다 일치하는 것을 볼 수 있습니다. 이는 모든 그래프 텐서를 캡처하기 위해 선택한 step입니다.

![bokeh-graph](./images/bokeh_graph.png)

In [ ]:
import numpy as np
from bokeh.models import ColumnDataSource, CustomJS, Div, HoverTool, HBar
from bokeh.models.glyphs import Circle, Line
from bokeh.plotting import figure, show

plot = figure(
    plot_height=400,
    plot_width=1400,
    x_range=(timestamps[0], timestamps[-1]),
    y_range=(-1, 110),
    tools="crosshair,xbox_select,pan,reset,save,xwheel_zoom",
)
x_range = plot.x_range

plot.xgrid.visible = False
plot.ygrid.visible = False

colors = np.where(step_num % 50 == 0, "yellow", "purple")

# pad framework metrics to match length of system metrics
pad = values.size - step_num.size
source = ColumnDataSource(
    data=dict(
        x=timestamps,
        y=values,
        left=np.pad(start_step, (0, pad)),
        right=np.pad(end_step, (0, pad)),
        color=np.pad(colors, (0, pad)),
    )
)


callback = CustomJS(
    args=dict(s1=source, div=Div(width=250, height=100, height_policy="fixed")),
    code="""
        console.log('Running CustomJS callback now.');
        var inds = s1.selected.indices;
        console.log(inds);
        var line = "<span style=float:left;clear:left;font_size=13px><b> Selected index range: [" + Math.min.apply(Math,inds) + "," + Math.max.apply(Math,inds) + "]</b></span>\\n";
        console.log(line)
        var text = div.text.concat(line);
        var lines = text.split("\\n")
        if (lines.length > 35)
            lines.shift();
        div.text = lines.join("\\n");""",
)

plot.js_on_event("selectiongeometry", callback)

line = Line(x="x", y="y", line_color="white")
circle = Circle(x="x", y="y", fill_alpha=0, line_width=0)
hbar = HBar(
    y=105, height=5, right="right", left="left", fill_color="color", line_cap="round", line_width=0
)


p = plot.add_glyph(source, line)
p = plot.add_glyph(source, circle)
p = plot.add_glyph(source, hbar)

# create tooltip for hover tool
hover = HoverTool(renderers=[p], tooltips=[("index", "$index"), ("(x,y)", "($x, $y)")])

plot.xaxis.axis_label = "Time in ms"
plot.yaxis.axis_label = "GPU Utilization"
plot.add_tools(hover)
show(plot, notebook_handle=True)